# inter-run comparisons of N contained in phytoplankton
    - NO full domain sum
    - S3
    - Sentry Shoal
    - Central Node
    - JDF

In [1]:
import netCDF4 as nc
import datetime as dt
import subprocess
import requests
import matplotlib.pyplot as plt
import cmocean
import numpy as np
import os
import re
import dateutil as dutil
from salishsea_tools import viz_tools, places
import glob
import pickle
import subprocess
import time
%matplotlib inline

In [2]:
import socket
socket.gethostname()

'salish.eos.ubc.ca'

In [3]:
dirname='spring2015_hiNH'#'spring2015_uzPref'#'spring2015_SMELTBFastSink'#'spring2015_KhT'
saveloc='/data/eolson/MEOPAR/SS36runs/calcFiles/comparePhytoN/'
dirnames=('spring2015_NewSink','spring2015_KhT','spring2015_uzoo','spring2015_uzoo2','spring2015_slowPP','spring2015_lowMuNano',
          'spring2015_SMELTBFastSink','spring2015_uzPref','spring2015_hiNH')
spaths={'spring2015_NewSink':'/data/eolson/results/MEOPAR/SS36runs/CedarRuns/spring2015_NewSink/',
    'spring2015_KhT':'/data/eolson/results/MEOPAR/SS36runs/CedarRuns/spring2015_KhT/',
    'spring2015_uzoo':'/data/eolson/results/MEOPAR/SS36runs/CedarRuns/spring2015_uzoo/',
    'spring2015_uzoo2':'/data/eolson/results/MEOPAR/SS36runs/CedarRuns/spring2015_uzoo2/',
    'spring2015_slowPP':'/data/eolson/results/MEOPAR/SS36runs/OrcinusRuns/spring2015_slowPP/',
    'spring2015_lowMuNano':'/data/eolson/results/MEOPAR/SS36runs/CedarRuns/spring2015_lowMuNano/',
    'spring2015_SMELTBFastSink':'/data/eolson/results/MEOPAR/SS36runs/OrcinusRuns/spring2015_SMELTBFastSink/',
    'spring2015_uzPref':'/data/eolson/results/MEOPAR/SS36runs/CedarRuns/spring2015_uzPref/',
    'spring2015_hiNH':'/data/eolson/results/MEOPAR/SS36runs/CedarRuns/spring2015_hiNH/'}
# whether e3t is stored in ptrc or not:
e3tp={'spring2015_NewSink':False,
    'spring2015_KhT':False,
    'spring2015_uzoo':True,
    'spring2015_uzoo2':True,
    'spring2015_slowPP':True,
    'spring2015_lowMuNano':False,
    'spring2015_SMELTBFastSink':True,
    'spring2015_uzPref':True,
    'spring2015_hiNH':True}
t0=dt.datetime(2015,2,6)
fdur=10 # length of each results file in days
spath=spaths[dirname]
fnum=len(glob.glob(spath+'*ptrc*'))#18 # number of results files per run
runlen=fdur*fnum # length of run in days
#stm=np.shape(tmask)

In [4]:
plist=['Sentry Shoal','S3','Central node','Central SJDF']
varNameDict={'Sentry Shoal':'SentryShoal', 'S3':'S3', 'Central node':'CentralNode', 'Central SJDF':'CentralSJDF','all':'All'}

In [5]:
def createAreaFile(tlen=1,meshpath='/ocean/eolson/MEOPAR/NEMO-forcing/grid/mesh_mask201702_noLPE.nc',
                  ptrcexpath='/data/eolson/results/MEOPAR/SS36runs/CedarRuns/spring2015_KhT/SalishSea_1h_20150206_20150804_ptrc_T_20150616-20150625.nc'):
    # meshpath: mesh_mask to base calculations on
    # ptrcexpath: sample model output file to grab dimensions from (must be consistent for nco tools to work)
    foutname=saveloc+'Area_'+str(tlen)+'.nc'
    if os.path.isfile(foutname):
        os.remove(foutname)
    fout=nc.Dataset(foutname, "w", format="NETCDF4")
    with nc.Dataset(ptrcexpath) as fex:
        for dname, the_dim in fex.dimensions.items():
            print(dname, len(the_dim))
            fout.createDimension(dname, len(the_dim) if not the_dim.isunlimited() else None)
    with nc.Dataset(meshpath) as fm:
        tmask=np.copy(fm.variables['tmask'])
        e1t=np.copy(fm.variables['e1t'])
        e2t=np.copy(fm.variables['e2t'])
        A3t=e1t*e2t*tmask
    if tlen>1:
        A=np.tile(A3t,(tlen,1,1,1))
    else:
        A=A3t
    outVar = fout.createVariable('A', 'float32', ('time_counter', 'deptht', 'y', 'x'),zlib=True)
    outVar[:] = A[:]
    fout.close()
    #p=subprocess.call('nccopy -s -4 -d4 '+foutname+' '+foutnameF,shell=True)
    #p=subprocess.call('rm '+foutname,shell=True)
    return foutname

In [6]:
Aloc='/data/eolson/MEOPAR/SS36runs/calcFiles/comparePhytoN/Area_240.nc'

In [7]:
fnames={'ptrc_T':dict(),'grid_T':dict(),'tempBase':dict()}
for ii in range(0,fnum):
    ts=(t0+dt.timedelta(days=ii*fdur)).strftime('%Y%m%d')
    te=(t0+dt.timedelta(days=((ii+1)*fdur-1))).strftime('%Y%m%d')
    fnames['ptrc_T'][ii]=glob.glob(spath+'*ptrc_T_'+ts+'-'+te+'.nc')[0]
    if not e3tp[dirname]:
        try:
            fnames['grid_T'][ii]=glob.glob(spath+'*e3t_'+ts+'-'+te+'.nc')[0]
        except:
            fnames['grid_T'][ii]=glob.glob(spath+'*grid_T_'+ts+'-'+te+'.nc')[0]
    fnames['tempBase'][ii]=saveloc+'temp/ptrc_'+dirname+'_'+ts+'-'+te

In [8]:
maxproc=3

In [9]:
pids=dict()
for ii in range(0,fnum):
    # copy ptrc
    f0=fnames['tempBase'][ii]+'.nc'
    if e3tp[dirname]:
        pids[ii]=subprocess.Popen('ncks -v diatoms,flagellates,ciliates,e3t,nitrate,silicon '+fnames['ptrc_T'][ii]+' '+f0, shell=True)
    else:
        pids[ii]=subprocess.Popen('ncks -v diatoms,flagellates,ciliates,nitrate,silicon '+fnames['ptrc_T'][ii]+' '+f0, shell=True)
    if ii%maxproc==(maxproc-1):
        pids[ii].wait() # wait for the last one
# check that no others are still running, wait for them
for ipid in pids.keys():
    while pids[ipid].poll() is None:
        time.sleep(30)

In [10]:
if not e3tp[dirname]:
    pids=dict()
    for ii in range(0,fnum):
        # copy ptrc
        f0=fnames['tempBase'][ii]+'.nc'
        # add e3t to ptrc
        pids[ii]=subprocess.Popen('ncks -A -v e3t '+fnames['grid_T'][ii]+' '+f0, shell=True)
        if ii%maxproc==(maxproc-1):
            pids[ii].wait() # wait for the last one
    # check that no others are still running, wait for them
    for ipid in pids.keys():
        while pids[ipid].poll() is None:
            time.sleep(30)

In [11]:
# extract phyto and e3t at locs
for pl in plist:
    pids=dict()
    for ii in range(0,fnum):
        f0=fnames['tempBase'][ii]+'.nc'
        fpl=fnames['tempBase'][ii]+varNameDict[pl]+'.nc'
        j,i=places.PLACES[pl]['NEMO grid ji']
        pids[ii]=subprocess.Popen('ncks -v diatoms,flagellates,ciliates,e3t,nitrate,silicon -d x,'+str(i)+' -d y,'+str(j)+' '+f0+' '+fpl, shell=True)
        if ii%maxproc==(maxproc-1):
            pids[ii].wait() # wait for the last one
    # check that no others are still running, wait for them
    for ipid in pids.keys():
        while pids[ipid].poll() is None:
            time.sleep(30)

In [12]:
pids=dict()
jj=0
for pl in plist:
    fpllist=list()
    f1=saveloc+'ts_'+dirname+'_'+varNameDict[pl]+'.nc'
    for ii in range(0,fnum):
        fpllist.append(fnames['tempBase'][ii]+varNameDict[pl]+'.nc')
    pids[jj]=subprocess.Popen('ncrcat '+' '.join(fpllist)+' '+f1, shell=True)
    jj+=1
pids[jj-1].wait() # wait for the last one
# check that no others are still running, wait for them
for ipid in pids.keys():
    while pids[ipid].poll() is None:
        time.sleep(30)